# librerias necesarias

In [20]:
import numpy as np
import itertools # para generar combinarion de elemntos de una lista

# ecuaciones

CONCEPTOS PARA EL ALGORITMO APRIORI

SUPPORT = cantidad item A / NRO de itemsets

CONFIDENCE(A -> B)
CONFIDENCE = cantidad de A y B / cantidad item A 

LIFT(A -> B)
LIFT = confidence(A -> B) / suport(B)

# cargar el dataSet

In [21]:
data = np.load('/Users/blablabla/usuario_alojado/SEMESTRE_II_2021/Mineria-de-datos/Actividad03/spotify.npy', allow_pickle=True)
data1 = data.item(0) # contiene los itemsets en una diccionario {0: [itemset1], 1: [itemset2],...}

# dataset de prueba para testear los modulos individuales
dataTest = {0:['Pan','Leche','Pañales'],
           1:['Pan','Pañales','Cerveza','Huevos'],
           2:['Leche','Pañales','Cerveza','Refresco','Café'],
           3:['Pan','Leche','Pañales','Cerveza'],
           4:['Pan','Refresco','Leche','Pañales']}

### modulo para genera nuevos super ItemSets

In [22]:
def candidateGet(itemSetsFrecuentes, length): #retorna una lista con los nuevos superItemSets 
    superItemSets = []
    for index, itemSet in enumerate(itemSetsFrecuentes): # rrecorremos cada itemset
        for j in range(index+1,len(itemSetsFrecuentes)): # rrecorre apartir del siguiente itemset 
            union = np.append(itemSet,itemSetsFrecuentes[j]) # mescla el itemset con todos los itemset que estan delante 
            union = list(set(union)) # elimina elementos repetidos
            union = sorted(union) # ordena los elementos
            #print(union)
            if len(union) == length and union not in superItemSets: # filtra los itemsets nuevos que complan con length
                superItemSets.append(union)
    return superItemSets

### modulo para podar los superItemSets que contengan subItemSet no frecuentes

In [23]:
def candidatePrune( newItemSets, itemSetsFrecuentes): # retorna un arreglo
    pruneList = []
    for itemSet in newItemSets: # rrecorremos cada itemset 
        cont = 0 # para validar que todos sus subItemSet sean frecuentes
        # combinar los elementos del itemset actual en grupos de tamaños len(itemSet)-1 (una unidad menor que el itemset)
        # para verificar que sus sub grupos sean frecuentes
        combinaciones = list(itertools.combinations(itemSet,len(itemSet)-1)) 
        #print(len(combinaciones), combinaciones)
        for element in combinaciones: # contamos cuantos de esas combinaciones estan en itemSetsFrecuentes 
            #print(element, list(element))
            if list(element) in itemSetsFrecuentes:
                cont += 1
        #print(cont)
        if cont == len(combinaciones): # verificamos que todas las combinaciones esten dentro de itemSetsFrecuentes
            pruneList.append(itemSet)
    return pruneList

### modulo para verificar si un subItemSet se encuentra dentro de un ItemSet original

In [24]:
def subset(Candidatos, itemSetOriginal): # retorna una lista de los candidatos que se encuentran en el ItemSetOriginal
    encontrados = []
    for candidato in Candidatos: # recorre todos los candidatos
        contador = 0 # para validar que se encontraron todos los items dentro del itemSetOriginal
        for item in candidato: # recorre todos los elementos del candidato
            if (item in itemSetOriginal): 
                contador += 1 # suma 1 si el item se encuentra dentro del itemSet original
        if contador == len(candidato): # comprueba que todos los items del candidato estan dentro del itemSet original
            encontrados.append(candidato)
    return encontrados

# modulo para extraer los itemSets frecuentes del data set

In [25]:
def get_frequent_itemsets(playlists, min_support): # min_supoort en el rango de [0,1]
    # VARIABLES 
    nroItemSets = len(playlists) # SUPPORT = cantidadItem / nroItemSets
    itemSetsFrecuentes = [] # lista que almacena los itemsets frecuentes 
    listSupport = [] # Lista que almacena los suports de los itemsets frecuentes
    
    # Encontrar los 1-itemsets frecuentes
    frecuencias = {} # acumula las item junsto a su nro de veces itemsets en los que aparece
    musicas = [] # guarda los items individuales
    noRepetidos = [] # es un filtro de la anterior lista, lista sin repetidos
    frecuentes = [] # 
    for i in playlists:
        for j in playlists[i]:
            musicas.append(j)
    noRepetidos = sorted(set(musicas))
    #print(noRepetidos)
    for i in noRepetidos:
        frecuencias[i] = musicas.count(i)
    for i in frecuencias:
        if frecuencias[i]/nroItemSets >= min_support:
            frecuentes.append([i])
            itemSetsFrecuentes.append([i])
            listSupport.append(frecuencias[i]/nroItemSets)
    # Bucle para encontrar n-itemsets frecuentes mientras existan.
    #VARIABLES
    print(frecuentes)
    k = 1
    FK = [] # lista de itemsets frecuentes de K-items 
    while (len(frecuentes) > 1):
        k += 1
        FK = candidateGet(frecuentes,k) # generar K-itemsets candidatos 
        FK = candidatePrune(FK, frecuentes) # podar candidatos que tengan sub-itemsets no frecuentes
        SUPPORT = [0]*len(FK) # creamos un arreglo que va a almacenar los support
        #print(playlists)
        for playlist in playlists: # bucle para calcular su SUPPORT y definir los nuevos K-itemsets frecuentes
            #print(playlist)
            encontrados = subset(FK, playlists[playlist])  # revisar que supergrupos de FK estan en la playlist
            for encontrado in encontrados:
                SUPPORT[FK.index(encontrado)] += 1
        frecuentes = []
        for index,candidato in enumerate(FK):
            #print(candidato)
            #print(SUPPORT[index]/nroItemSets)
            if (SUPPORT[index]/nroItemSets >= min_support):
                itemSetsFrecuentes.append(candidato)
                listSupport.append(SUPPORT[index]/nroItemSets)
                frecuentes.append(candidato)
    return itemSetsFrecuentes, listSupport
    
        

### generación de los itemsets frecuentes

In [ ]:
frecuentes, Supports = get_frequent_itemsets(data1,0.05)
print(frecuentes)
print(Supports)

### modulo para generar nuevas reglas

In [8]:
def rulesCandidateGen( rules, lengt):
    superItemSets = []
    for index, itemSet in enumerate(rules): # recorre todos las reglas actuales
        for j in range(index,len(rules)): # recorre las reglas desde la siguiente posicion
            union = np.append(itemSet,rules[j]) # une los elementos de la Regla actual con una de las reglas siguientes
            union = set(union) # se eliminan elementos repetidos
            union = sorted(union) # se ordenan los elementos
            if len(union) == lengt and union not in superItemSets: # se verifica que la longitud cumpla con lengt
                superItemSets.append(union)
    return superItemSets

### modulo para podar reglas 

In [9]:
def rulesCandidatePrune( candidatos, subCandidatos):
    candidatePrune = []
    for candidato in candidatos: # recorre todos los candidatos
        cont = 0 # contador para verificar que todos los subConjuntos de candidato sean reglas que superan el confidence
        if (len(candidato)-1 == 0): # en caso de que las reglas actuales (candidatos) tengan un solo elementos como antecedente
            subCombinaciones = candidatos
        else: # en caso de que las reglas actules (candidatos) tengan mas de 1 elemento como antecedente
            subCombinaciones = list(itertools.combinations(candidato, len(candidato)-1))
        for item in subCombinaciones: # contamos cuantos de sus subconjuntos son reglas que superan el confidence
            if list(item) in subCandidatos:
                cont += 1
        if cont == len(subCombinaciones): # ferificamos que todos sus subconjutnos sean reglas validas
            candidatePrune.append(candidato)
    return candidatePrune

# modulo para generar las reglas

In [10]:
def generate_association_rules(frequent_itemsets, supports, confidence = 0, lift = 0):
    # VARIABLES
    rules = [] # almacena las reglas que superen el confidence y ordenadas por el lift
    _confi = [] # almacena el confidence de las reglas
    _lift = [] # almacena el lift de las reglas
    listPoda = [] # almacena los subconjuntos que no superen el confidence
    
    #
    for index, itemset in enumerate(frequent_itemsets):
        # convertimos los elementos de itemset en listas
        newItemset = [[i] for i in itemset]
        k = len(newItemset) # tamaño del itemset
        m = 1 # tamaño inicial del consecuente de la regla
        H = newItemset # inicializacion de la lista que almacenara las reglas generadas de itemset 
        #print(k, itemset)
        while k > m:
            candidatos = rulesCandidateGen(H, m) # generamos nuevas reglas 
            candidatos = rulesCandidatePrune( candidatos, H) # podamos algunas reglas
            H = []
            for candidato in candidatos:
                _support = frequent_itemsets.index(candidato) # calculamos el indice del support para el candidato
                conf = supports[index] / supports[_support]
                if conf >= confidence:
                    # generar la regla que tiene la forma de [[antecedente],[precedente]]
                    antecedente = candidato
                    precedente  = list(sorted(set(itemset).difference(set(candidato))))
                    #print("precedente: ",precedente)
                    rules.append([antecedente,precedente])
                    _confi.append(conf)
                    # Calcular el lift
                    lift = conf / supports[frequent_itemsets.index(precedente)]
                    _lift.append(lift)
                    H.append(candidato)
            m += 1
    
    return rules, _lift
        

### generacion de reglas

In [ ]:
reglas, listLift = generate_association_rules(frecuentes, Supports, confidence = 0.5, lift = 1.2)
for index, regla in enumerate(reglas):
    print(regla, listLift[index])